# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

**There are 17 Marketing Campaigns that the data is representing**

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')
pd.set_option('display.max_rows', None)

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
# PLOT VALUES THAT ARE MISSING IF MISSING
if df.isnull().any().sum() > 0:
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.xticks(rotation=45, ha='right')
    plt.show()
    plt.close()

else:
    print("No missing values")




No missing values


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

**Objective Overview**: To predict whether a client will subscribe to a term deposit at a bank.

- **For the Bank**: The bank wants to understand and predict client behavior towards subscriptions by insights from effective target marketing strategies and in turn increase the success rate of the bank’s marketing campaigns.
- **For the Clients**: The clients want a more customized marketing experience catered to themselves. This ensures that clients receive offers that are more likely to align with their needs and financial goals.

The business objective is to leverage the data to build a predictive model that can identify potential clients who are more likely to subscribe to a term deposit.

---
### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [6]:
df.rename(columns={'y': 'subscribed'}, inplace=True)

In [7]:
# Selecting the bank information features (columns 1 - 7) and the target column
features = df.iloc[:, 0:7]
print(features.columns)
target = df['subscribed']

# Encoding categorical variables using OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, )
encoded_features = encoder.fit_transform(features.select_dtypes(include=['object']))

# Creating a DataFrame for the encoded features
encoded_feature_names = encoder.get_feature_names_out(features.select_dtypes(include=['object']).columns)
encoded_features_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)

# Combining encoded categorical features with numerical features
final_features = pd.concat([features.select_dtypes(exclude=['object']), encoded_features_df], axis=1)

# Encoding the target variable
target = target.map({'yes': 1, 'no': 0})

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan'], dtype='object')


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [8]:
# Split the data into a training set (80%) and a test set (20%)
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(final_features, target, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((32950, 34), (8238, 34), (32950,), (8238,))

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [9]:
# Calculate the baseline accuracy by predicting the most frequent class
baseline_accuracy = y_train.value_counts(normalize=True).max()
baseline_accuracy

0.887556904400607

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [10]:
# Creating the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)

# Fitting the model to the training data
logistic_model.fit(X_train, y_train)

# Making predictions on the test data
y_pred = logistic_model.predict(X_test)

# Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8865015780529255

### Problem 9: Score the Model

What is the accuracy of your model?

In [11]:
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy: {accuracy:.2%}")

Logistic Regression Model Accuracy: 88.65%


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [15]:
# List to store results
results = []

# List of models to evaluate
models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('KNN', KNeighborsClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('SVM', SVC())
]

# Fit and evaluate each model
for name, model in models:
    try:
        
        # Convert data to dense format for KNN if necessary
        if name == 'KNN':
            X_train = X_train.to_numpy() if not isinstance(X_train, np.ndarray) else X_train
            X_test = X_test.to_numpy() if not isinstance(X_test, np.ndarray) else X_test

            
        # 
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time

        train_accuracy = accuracy_score(y_train, model.predict(X_train))
        test_accuracy = accuracy_score(y_test, model.predict(X_test))

        results.append({
            'Model': name,
            'Train Time': train_time,
            'Train Accuracy': train_accuracy,
            'Test Accuracy': test_accuracy
        })
    except Exception as e:
        print(f"Error with model {name}: {e}")


# Create DataFrame from the results
results_df = pd.DataFrame(results)

# Display the results
print(results_df)

                 Model  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.409377        0.887557       0.886502
1                  KNN    0.004192        0.890015       0.873513
2        Decision Tree    0.081672        0.916601       0.861617
3                  SVM    8.604689        0.887557       0.886502


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [17]:
# Define the models and their hyperparameter grids
model_params = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs'],
            'max_iter': range(1000, 5000, 1000)
        }
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 11, 19],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_leaf': [1, 2, 4, 6]
        }
    },
    'SVM': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto'],
            'kernel': ['rbf', 'linear']
        }
    }
}

In [ ]:
# Perform Grid Search
grid_results = []
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    grid_results.append({
        'Model': model_name,
        'Best Parameters': clf.best_params_,
        'Test Accuracy': clf.score(X_test, y_test)
    })
    print(f"Finished Finished GridSearch on: {model_name}\n\
          Appending results to the list.")

Finished searching model: Logistic Regression
          Appending results to the list.
Finished searching model: KNN
          Appending results to the list.
Finished searching model: Decision Tree
          Appending results to the list.


In [ ]:
grid_results = pd.DataFrame(grid_results)

##### Questions